# Overview

This "DataCleaning_1" code takes the initial data, including weekly play-by-play and frame-by-frame (0.1 seconds), and creates week 1-9 dataframes titled 'sorted_df_1', 'sorted_df_2', etc. This dataframe includes the following features:

- A) Only includes running plays where a single handoff takes place for simplicity of the analysis (remove few plays where multiple handoffs take place)
- B) The play-by-play data spans from the frame where the handoff occurs and ends when a certain event occurs (fumble, out of bounds, touchdown, or tackle), and includes all 22 players data in relation to the ball carrier
- C) Player location and attributes for each player and the ball carrier by row, including the distance and angle to the ball carrier, and the player positions simplified to one of: DL, LB, DB, OL, TE, QB, SK (skill for RB/WR)
- D) Rank the players for each frame of a play in the following manner: 1) ball carrier, 2-8) 7 closest defenders to the ball carrier, 9-15) 7 closest offensive players to the ball carrier (besides QB), 16-22) remaining players distance to the ball carrier
- E) Saved as a .pkl file in Google Drive to be used in "DataCleaning_2"



# Import Packages and Files

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
games = pd.read_csv("games.csv")
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
tackles = pd.read_csv("tackles.csv")
tracking_week_1 = pd.read_csv("tracking_week_1.csv")
tracking_week_2 = pd.read_csv("tracking_week_2.csv")
tracking_week_3 = pd.read_csv("tracking_week_3.csv")
tracking_week_4 = pd.read_csv("tracking_week_4.csv")
tracking_week_5 = pd.read_csv("tracking_week_5.csv")
tracking_week_6 = pd.read_csv("tracking_week_6.csv")
tracking_week_7 = pd.read_csv("tracking_week_7.csv")
tracking_week_8 = pd.read_csv("tracking_week_8.csv")
tracking_week_9 = pd.read_csv("tracking_week_9.csv")

### Data

In [9]:
print(f"{games.shape[0]} rows and {games.shape[1]} columns")
games.head()

136 rows and 9 columns


,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23


In [10]:
print(f"{players.shape[0]} rows and {players.shape[1]} columns")
players.head()

1683 rows and 7 columns


,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [14]:
print(f"{plays.shape[0]} rows and {plays.shape[1]} columns")
plays.head(5).T

12486 rows and 35 columns


,0,1,2,3,4
gameId,2022100908,2022091103,2022091111,2022100212,2022091900
playId,3537,3126,1148,2007,1372
ballCarrierId,48723,52457,42547,46461,47857
ballCarrierDisplayName,Parker Hesse,Chase Claypool,Darren Waller,Mike Boone,Devin Singletary
playDescription,(7:52) (Shotgun) M.Mariota pass short middle t...,(7:38) (Shotgun) C.Claypool right end to PIT 3...,(8:57) D.Carr pass short middle to D.Waller to...,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,(8:33) D.Singletary right guard to TEN 32 for ...
quarter,4,4,2,3,2
down,1,1,2,2,1
yardsToGo,10,10,5,10,10
possessionTeam,ATL,PIT,LV,DEN,BUF
defensiveTeam,TB,CIN,LAC,LV,TEN


In [12]:
print(f"{tackles.shape[0]} rows and {tackles.shape[1]} columns")
tackles.head()

17426 rows and 7 columns


,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0


In [13]:
print(f"{tracking_week_1.shape[0]} rows and {tracking_week_1.shape[1]} columns")
tracking_week_1.head()

1407439 rows and 17 columns


,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN


**A) Only includes running plays where a single handoff takes place for simplicity of the analysis (remove few plays where multiple handoffs take place)**

### Find the Beginning and End Frames for Run Plays

This section I'm looking to take the play-by-play tracking data and do the following:

- filter for plays that include a single handoff (remove the few with multiple handoffs for simplicity reasons)
- find the frames where the handoff takes place as well as an "ending frame" where a fumble, tackle, touchdown, or going out of bounds takes place.

In [15]:
# Find the play event where the ball is handed off

p = plays[['gameId', 'playId', 'passResult', 'playResult', 'quarter', 'down', 'playDescription', 'ballCarrierId', 'ballCarrierDisplayName']]
p_p = p.merge(players[['nflId', 'position']], left_on='ballCarrierId', right_on='nflId', how='inner').drop(columns=['nflId'])
p_p_rushes = p_p[p_p['passResult'].isna()].drop(columns=['passResult'])

a1 = tracking_week_1[(tracking_week_1['displayName'] == 'football') & ((tracking_week_1['event'] == 'handoff'))]
a2 = tracking_week_2[(tracking_week_2['displayName'] == 'football') & ((tracking_week_2['event'] == 'handoff'))]
a3 = tracking_week_3[(tracking_week_3['displayName'] == 'football') & ((tracking_week_3['event'] == 'handoff'))]
a4 = tracking_week_4[(tracking_week_4['displayName'] == 'football') & ((tracking_week_4['event'] == 'handoff'))]
a5 = tracking_week_5[(tracking_week_5['displayName'] == 'football') & ((tracking_week_5['event'] == 'handoff'))]
a6 = tracking_week_6[(tracking_week_6['displayName'] == 'football') & ((tracking_week_6['event'] == 'handoff'))]
a7 = tracking_week_7[(tracking_week_7['displayName'] == 'football') & ((tracking_week_7['event'] == 'handoff'))]
a8 = tracking_week_8[(tracking_week_8['displayName'] == 'football') & ((tracking_week_8['event'] == 'handoff'))]
a9 = tracking_week_9[(tracking_week_9['displayName'] == 'football') & ((tracking_week_9['event'] == 'handoff'))]

# Group by 'gameId' and 'playId', then apply a lambda function to count 'handoff' occurrences
handoff_counts_1 = a1.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
# Find game/play combinations with more than one 'handoff' event
multiple_handoffs_1 = handoff_counts_1[handoff_counts_1 > 1].reset_index()
# Rename the columns for clarity
multiple_handoffs_1.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_2 = a2.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_2 = handoff_counts_2[handoff_counts_2 > 1].reset_index()
multiple_handoffs_2.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_3 = a3.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_3 = handoff_counts_3[handoff_counts_3 > 1].reset_index()
multiple_handoffs_3.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_4 = a4.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_4 = handoff_counts_4[handoff_counts_4 > 1].reset_index()
multiple_handoffs_4.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_5 = a5.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_5 = handoff_counts_5[handoff_counts_5 > 1].reset_index()
multiple_handoffs_5.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_6 = a6.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_6 = handoff_counts_6[handoff_counts_6 > 1].reset_index()
multiple_handoffs_6.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_7 = a7.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_7 = handoff_counts_7[handoff_counts_7 > 1].reset_index()
multiple_handoffs_7.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_8 = a8.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_8 = handoff_counts_8[handoff_counts_8 > 1].reset_index()
multiple_handoffs_8.columns = ['gameId', 'playId', 'handoff_count']

handoff_counts_9 = a9.groupby(['gameId', 'playId'])['event'].apply(lambda events: ((events == 'handoff')).sum())
multiple_handoffs_9 = handoff_counts_9[handoff_counts_9 > 1].reset_index()
multiple_handoffs_9.columns = ['gameId', 'playId', 'handoff_count']

In [16]:
# Get data so it begins with ball carrier getting handoff, ends with a fumble, tackle, touchdown, or going out of bounds
df_filtered_1 = a1.merge(multiple_handoffs_1, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_1 = df_filtered_1[df_filtered_1['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_2 = a2.merge(multiple_handoffs_2, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_2 = df_filtered_2[df_filtered_2['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_3 = a3.merge(multiple_handoffs_3, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_3 = df_filtered_3[df_filtered_3['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_4 = a4.merge(multiple_handoffs_4, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_4 = df_filtered_4[df_filtered_4['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_5 = a5.merge(multiple_handoffs_5, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_5 = df_filtered_5[df_filtered_5['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_6 = a6.merge(multiple_handoffs_6, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_6 = df_filtered_6[df_filtered_6['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_7 = a7.merge(multiple_handoffs_7, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_7 = df_filtered_7[df_filtered_7['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_8 = a8.merge(multiple_handoffs_8, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_8 = df_filtered_8[df_filtered_8['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

df_filtered_9 = a9.merge(multiple_handoffs_9, on=['gameId', 'playId'], how='left', indicator=True)
df_filtered_9 = df_filtered_9[df_filtered_9['_merge'] == 'left_only'].drop(columns='_merge').reset_index(drop=True)

w1_rushes = p_p_rushes.merge(df_filtered_1[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w2_rushes = p_p_rushes.merge(df_filtered_2[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w3_rushes = p_p_rushes.merge(df_filtered_3[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w4_rushes = p_p_rushes.merge(df_filtered_4[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w5_rushes = p_p_rushes.merge(df_filtered_5[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w6_rushes = p_p_rushes.merge(df_filtered_6[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w7_rushes = p_p_rushes.merge(df_filtered_7[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w8_rushes = p_p_rushes.merge(df_filtered_8[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
w9_rushes = p_p_rushes.merge(df_filtered_9[['gameId', 'playId', 'playDirection']], left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')

filtered_events = ['tackle', 'out_of_bounds', 'touchdown', 'fumble']
filtered_plays_1 = tracking_week_1[(tracking_week_1['displayName'] == 'football') & (tracking_week_1['event'].isin(filtered_events))]
filtered_plays_2 = tracking_week_2[(tracking_week_2['displayName'] == 'football') & (tracking_week_2['event'].isin(filtered_events))]
filtered_plays_3 = tracking_week_3[(tracking_week_3['displayName'] == 'football') & (tracking_week_3['event'].isin(filtered_events))]
filtered_plays_4 = tracking_week_4[(tracking_week_4['displayName'] == 'football') & (tracking_week_4['event'].isin(filtered_events))]
filtered_plays_5 = tracking_week_5[(tracking_week_5['displayName'] == 'football') & (tracking_week_5['event'].isin(filtered_events))]
filtered_plays_6 = tracking_week_6[(tracking_week_6['displayName'] == 'football') & (tracking_week_6['event'].isin(filtered_events))]
filtered_plays_7 = tracking_week_7[(tracking_week_7['displayName'] == 'football') & (tracking_week_7['event'].isin(filtered_events))]
filtered_plays_8 = tracking_week_8[(tracking_week_8['displayName'] == 'football') & (tracking_week_8['event'].isin(filtered_events))]
filtered_plays_9 = tracking_week_9[(tracking_week_9['displayName'] == 'football') & (tracking_week_9['event'].isin(filtered_events))]

w_a1 = w1_rushes[['gameId', 'playId']].merge(filtered_plays_1, on=['gameId', 'playId'], how='inner')
w_a2 = w2_rushes[['gameId', 'playId']].merge(filtered_plays_2, on=['gameId', 'playId'], how='inner')
w_a3 = w3_rushes[['gameId', 'playId']].merge(filtered_plays_3, on=['gameId', 'playId'], how='inner')
w_a4 = w4_rushes[['gameId', 'playId']].merge(filtered_plays_4, on=['gameId', 'playId'], how='inner')
w_a5 = w5_rushes[['gameId', 'playId']].merge(filtered_plays_5, on=['gameId', 'playId'], how='inner')
w_a6 = w6_rushes[['gameId', 'playId']].merge(filtered_plays_6, on=['gameId', 'playId'], how='inner')
w_a7 = w7_rushes[['gameId', 'playId']].merge(filtered_plays_7, on=['gameId', 'playId'], how='inner')
w_a8 = w8_rushes[['gameId', 'playId']].merge(filtered_plays_8, on=['gameId', 'playId'], how='inner')
w_a9 = w9_rushes[['gameId', 'playId']].merge(filtered_plays_9, on=['gameId', 'playId'], how='inner')

**- B) The play-by-play data spans from the frame where the handoff occurs and ends when a certain event occurs (fumble, out of bounds, touchdown, or tackle), and includes all 22 players data in relation to the ball carrier**

In [19]:
event_priority = ['fumble', 'out_of_bounds', 'touchdown', 'tackle']

# custom function
def find_prioritized_event(group, event_priority):
  for event in event_priority:
    if event in group['event'].values:
      return group.loc[group['event'] == event].iloc[0]
  return group.iloc[0]

In [20]:
# Group by 'gameId' and 'playId' and apply the custom function
priority_frames_1 = w_a1.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_2 = w_a2.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_3 = w_a3.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_4 = w_a4.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_5 = w_a5.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_6 = w_a6.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_7 = w_a7.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_8 = w_a8.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)
priority_frames_9 = w_a9.groupby(['gameId', 'playId']).apply(find_prioritized_event, event_priority=event_priority).reset_index(drop=True)

params_df_1 = a1[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_1[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_2 = a2[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_2[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_3 = a3[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_3[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_4 = a4[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_4[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_5 = a5[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_5[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_6 = a6[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_6[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_7 = a7[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_7[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_8 = a8[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_8[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')
params_df_9 = a9[['gameId', 'playId', 'frameId', 'event']].merge(priority_frames_9[['gameId', 'playId', 'frameId', 'event']], on=['gameId', 'playId'], how='inner')

w1_rushes_params = params_df_1.merge(w1_rushes, on=['gameId', 'playId'], how= 'inner')
w2_rushes_params = params_df_2.merge(w2_rushes, on=['gameId', 'playId'], how= 'inner')
w3_rushes_params = params_df_3.merge(w3_rushes, on=['gameId', 'playId'], how= 'inner')
w4_rushes_params = params_df_4.merge(w4_rushes, on=['gameId', 'playId'], how= 'inner')
w5_rushes_params = params_df_5.merge(w5_rushes, on=['gameId', 'playId'], how= 'inner')
w6_rushes_params = params_df_6.merge(w6_rushes, on=['gameId', 'playId'], how= 'inner')
w7_rushes_params = params_df_7.merge(w7_rushes, on=['gameId', 'playId'], how= 'inner')
w8_rushes_params = params_df_8.merge(w8_rushes, on=['gameId', 'playId'], how= 'inner')
w9_rushes_params = params_df_9.merge(w9_rushes, on=['gameId', 'playId'], how= 'inner')


t_w1 = tracking_week_1[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w2 = tracking_week_2[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w3 = tracking_week_3[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w4 = tracking_week_4[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w5 = tracking_week_5[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w6 = tracking_week_6[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w7 = tracking_week_7[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w8 = tracking_week_8[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]
t_w9 = tracking_week_9[['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'event']]

### Use SQL query to create a dataframe where the frames are between the handoff and final event, and includes the ball carrier info

In [22]:
import sqlite3

# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w1.to_sql('df1', conn, index=False, if_exists='replace')
w1_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_1 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [23]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w2.to_sql('df1', conn, index=False, if_exists='replace')
w2_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_2 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [24]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w3.to_sql('df1', conn, index=False, if_exists='replace')
w3_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_3 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [25]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w4.to_sql('df1', conn, index=False, if_exists='replace')
w4_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_4 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [26]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w5.to_sql('df1', conn, index=False, if_exists='replace')
w5_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_5 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [27]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w6.to_sql('df1', conn, index=False, if_exists='replace')
w6_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_6 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [28]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w7.to_sql('df1', conn, index=False, if_exists='replace')
w7_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_7 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [29]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w8.to_sql('df1', conn, index=False, if_exists='replace')
w8_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_8 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [30]:
# Connect to a SQLite database (in-memory)
conn = sqlite3.connect(':memory:')

# Write the dataframes to the SQLite database
t_w9.to_sql('df1', conn, index=False, if_exists='replace')
w9_rushes_params.to_sql('df2', conn, index=False, if_exists='replace')

# Perform the SQL query to handle the conditional join
query = """
SELECT *
FROM df1
JOIN df2 ON df1.gameId = df2.gameId AND df1.playId = df2.playId AND df1.nflId = df2.ballCarrierId
WHERE df2.frameId_x <= df1.frameId AND df2.frameId_y >= df1.frameId
"""

# Read the query result into a new Pandas DataFrame
filtered_df_9 = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()

In [33]:
finished_play_loc_1 = filtered_df_1[filtered_df_1['frameId_y'] == filtered_df_1['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_2 = filtered_df_2[filtered_df_2['frameId_y'] == filtered_df_2['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_3 = filtered_df_3[filtered_df_3['frameId_y'] == filtered_df_3['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_4 = filtered_df_4[filtered_df_4['frameId_y'] == filtered_df_4['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_5 = filtered_df_5[filtered_df_5['frameId_y'] == filtered_df_5['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_6 = filtered_df_6[filtered_df_6['frameId_y'] == filtered_df_6['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_7 = filtered_df_7[filtered_df_7['frameId_y'] == filtered_df_7['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_8 = filtered_df_8[filtered_df_8['frameId_y'] == filtered_df_8['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T
finished_play_loc_9 = filtered_df_9[filtered_df_9['frameId_y'] == filtered_df_9['frameId']][['gameId', 'playId', 'x', 'y', 'playDirection', 'frameId', 'event']].T.drop_duplicates().T

finished_play_loc_1.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_2.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_3.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_4.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_5.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_6.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_7.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_8.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)
finished_play_loc_9.rename(columns={'x': 'final_x', 'y': 'final_y'}, inplace=True)

filtered_df_1 = filtered_df_1.T.drop_duplicates().T
filtered_df_2 = filtered_df_2.T.drop_duplicates().T
filtered_df_3 = filtered_df_3.T.drop_duplicates().T
filtered_df_4 = filtered_df_4.T.drop_duplicates().T
filtered_df_5 = filtered_df_5.T.drop_duplicates().T
filtered_df_6 = filtered_df_6.T.drop_duplicates().T
filtered_df_7 = filtered_df_7.T.drop_duplicates().T
filtered_df_8 = filtered_df_8.T.drop_duplicates().T
filtered_df_9 = filtered_df_9.T.drop_duplicates().T

filtered_df_final_x_1 = filtered_df_1.merge(finished_play_loc_1[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_2 = filtered_df_2.merge(finished_play_loc_2[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_3 = filtered_df_3.merge(finished_play_loc_3[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_4 = filtered_df_4.merge(finished_play_loc_4[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_5 = filtered_df_5.merge(finished_play_loc_5[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_6 = filtered_df_6.merge(finished_play_loc_6[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_7 = filtered_df_7.merge(finished_play_loc_7[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_8 = filtered_df_8.merge(finished_play_loc_8[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')
filtered_df_final_x_9 = filtered_df_9.merge(finished_play_loc_9[['gameId', 'playId', 'final_x', 'final_y']], on=['gameId', 'playId'], how='inner')

filtered_df_final_x_1['adjusted_x'] = np.where(
    (filtered_df_final_x_1['event'] == 'touchdown') & (filtered_df_final_x_1['x'] < 50), 10,
    np.where((filtered_df_final_x_1['event'] == 'touchdown') & (filtered_df_final_x_1['x'] > 50), 110, filtered_df_final_x_1['x'])
)

filtered_df_final_x_2['adjusted_x'] = np.where(
    (filtered_df_final_x_2['event'] == 'touchdown') & (filtered_df_final_x_2['x'] < 50), 10,
    np.where((filtered_df_final_x_2['event'] == 'touchdown') & (filtered_df_final_x_2['x'] > 50), 110, filtered_df_final_x_2['x'])
)

filtered_df_final_x_3['adjusted_x'] = np.where(
    (filtered_df_final_x_3['event'] == 'touchdown') & (filtered_df_final_x_3['x'] < 50), 10,
    np.where((filtered_df_final_x_3['event'] == 'touchdown') & (filtered_df_final_x_3['x'] > 50), 110, filtered_df_final_x_3['x'])
)

filtered_df_final_x_4['adjusted_x'] = np.where(
    (filtered_df_final_x_4['event'] == 'touchdown') & (filtered_df_final_x_4['x'] < 50), 10,
    np.where((filtered_df_final_x_4['event'] == 'touchdown') & (filtered_df_final_x_4['x'] > 50), 110, filtered_df_final_x_4['x'])
)

filtered_df_final_x_5['adjusted_x'] = np.where(
    (filtered_df_final_x_5['event'] == 'touchdown') & (filtered_df_final_x_5['x'] < 50), 10,
    np.where((filtered_df_final_x_5['event'] == 'touchdown') & (filtered_df_final_x_5['x'] > 50), 110, filtered_df_final_x_5['x'])
)

filtered_df_final_x_6['adjusted_x'] = np.where(
    (filtered_df_final_x_6['event'] == 'touchdown') & (filtered_df_final_x_6['x'] < 50), 10,
    np.where((filtered_df_final_x_6['event'] == 'touchdown') & (filtered_df_final_x_6['x'] > 50), 110, filtered_df_final_x_6['x'])
)

filtered_df_final_x_7['adjusted_x'] = np.where(
    (filtered_df_final_x_7['event'] == 'touchdown') & (filtered_df_final_x_7['x'] < 50), 10,
    np.where((filtered_df_final_x_7['event'] == 'touchdown') & (filtered_df_final_x_7['x'] > 50), 110, filtered_df_final_x_7['x'])
)

filtered_df_final_x_8['adjusted_x'] = np.where(
    (filtered_df_final_x_8['event'] == 'touchdown') & (filtered_df_final_x_8['x'] < 50), 10,
    np.where((filtered_df_final_x_8['event'] == 'touchdown') & (filtered_df_final_x_8['x'] > 50), 110, filtered_df_final_x_8['x'])
)

filtered_df_final_x_9['adjusted_x'] = np.where(
    (filtered_df_final_x_9['event'] == 'touchdown') & (filtered_df_final_x_9['x'] < 50), 10,
    np.where((filtered_df_final_x_9['event'] == 'touchdown') & (filtered_df_final_x_9['x'] > 50), 110, filtered_df_final_x_9['x'])
)


filtered_df_final_x_1['max_distance']  = np.where(filtered_df_final_x_1['playDirection'] == 'left', filtered_df_final_x_1['adjusted_x'] - 10, 110 - filtered_df_final_x_1['adjusted_x'])
filtered_df_final_x_2['max_distance']  = np.where(filtered_df_final_x_2['playDirection'] == 'left', filtered_df_final_x_2['adjusted_x'] - 10, 110 - filtered_df_final_x_2['adjusted_x'])
filtered_df_final_x_3['max_distance']  = np.where(filtered_df_final_x_3['playDirection'] == 'left', filtered_df_final_x_3['adjusted_x'] - 10, 110 - filtered_df_final_x_3['adjusted_x'])
filtered_df_final_x_4['max_distance']  = np.where(filtered_df_final_x_4['playDirection'] == 'left', filtered_df_final_x_4['adjusted_x'] - 10, 110 - filtered_df_final_x_4['adjusted_x'])
filtered_df_final_x_5['max_distance']  = np.where(filtered_df_final_x_5['playDirection'] == 'left', filtered_df_final_x_5['adjusted_x'] - 10, 110 - filtered_df_final_x_5['adjusted_x'])
filtered_df_final_x_6['max_distance']  = np.where(filtered_df_final_x_6['playDirection'] == 'left', filtered_df_final_x_6['adjusted_x'] - 10, 110 - filtered_df_final_x_6['adjusted_x'])
filtered_df_final_x_7['max_distance']  = np.where(filtered_df_final_x_7['playDirection'] == 'left', filtered_df_final_x_7['adjusted_x'] - 10, 110 - filtered_df_final_x_7['adjusted_x'])
filtered_df_final_x_8['max_distance']  = np.where(filtered_df_final_x_8['playDirection'] == 'left', filtered_df_final_x_8['adjusted_x'] - 10, 110 - filtered_df_final_x_8['adjusted_x'])
filtered_df_final_x_9['max_distance']  = np.where(filtered_df_final_x_9['playDirection'] == 'left', filtered_df_final_x_9['adjusted_x'] - 10, 110 - filtered_df_final_x_9['adjusted_x'])

**- C) Player location and attributes for each player and the ball carrier by row, including the distance and angle to the ball carrier, and the player positions simplified to one of: DL, LB, DB, OL, TE, QB, SK (skill for RB/WR)**

In [35]:
# calculate the ball carrier's actual distance from where the play ends to the current position

filtered_df_final_x_1['actual_dist_from_final'] = np.where(filtered_df_final_x_1['playDirection'] == 'left', filtered_df_final_x_1['adjusted_x'] - filtered_df_final_x_1['final_x'], filtered_df_final_x_1['final_x'] - filtered_df_final_x_1['adjusted_x'])
filtered_df_final_x_2['actual_dist_from_final'] = np.where(filtered_df_final_x_2['playDirection'] == 'left', filtered_df_final_x_2['adjusted_x'] - filtered_df_final_x_2['final_x'], filtered_df_final_x_2['final_x'] - filtered_df_final_x_2['adjusted_x'])
filtered_df_final_x_3['actual_dist_from_final'] = np.where(filtered_df_final_x_3['playDirection'] == 'left', filtered_df_final_x_3['adjusted_x'] - filtered_df_final_x_3['final_x'], filtered_df_final_x_3['final_x'] - filtered_df_final_x_3['adjusted_x'])
filtered_df_final_x_4['actual_dist_from_final'] = np.where(filtered_df_final_x_4['playDirection'] == 'left', filtered_df_final_x_4['adjusted_x'] - filtered_df_final_x_4['final_x'], filtered_df_final_x_4['final_x'] - filtered_df_final_x_4['adjusted_x'])
filtered_df_final_x_5['actual_dist_from_final'] = np.where(filtered_df_final_x_5['playDirection'] == 'left', filtered_df_final_x_5['adjusted_x'] - filtered_df_final_x_5['final_x'], filtered_df_final_x_5['final_x'] - filtered_df_final_x_5['adjusted_x'])
filtered_df_final_x_6['actual_dist_from_final'] = np.where(filtered_df_final_x_6['playDirection'] == 'left', filtered_df_final_x_6['adjusted_x'] - filtered_df_final_x_6['final_x'], filtered_df_final_x_6['final_x'] - filtered_df_final_x_6['adjusted_x'])
filtered_df_final_x_7['actual_dist_from_final'] = np.where(filtered_df_final_x_7['playDirection'] == 'left', filtered_df_final_x_7['adjusted_x'] - filtered_df_final_x_7['final_x'], filtered_df_final_x_7['final_x'] - filtered_df_final_x_7['adjusted_x'])
filtered_df_final_x_8['actual_dist_from_final'] = np.where(filtered_df_final_x_8['playDirection'] == 'left', filtered_df_final_x_8['adjusted_x'] - filtered_df_final_x_8['final_x'], filtered_df_final_x_8['final_x'] - filtered_df_final_x_8['adjusted_x'])
filtered_df_final_x_9['actual_dist_from_final'] = np.where(filtered_df_final_x_9['playDirection'] == 'left', filtered_df_final_x_9['adjusted_x'] - filtered_df_final_x_9['final_x'], filtered_df_final_x_9['final_x'] - filtered_df_final_x_9['adjusted_x'])

handoffs_1 = w1_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_2 = w2_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_3 = w3_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_4 = w4_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_5 = w5_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_6 = w6_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_7 = w7_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_8 = w8_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]
handoffs_9 = w9_rushes_params[['gameId', 'playId', 'frameId_x', 'event_x', 'frameId_y', 'event_y', 'ballCarrierId', 'ballCarrierDisplayName', 'playDirection']]

handoffs_xy_1 = handoffs_1.merge(filtered_df_final_x_1[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_2 = handoffs_2.merge(filtered_df_final_x_2[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_3 = handoffs_3.merge(filtered_df_final_x_3[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_4 = handoffs_4.merge(filtered_df_final_x_4[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_5 = handoffs_5.merge(filtered_df_final_x_5[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_6 = handoffs_6.merge(filtered_df_final_x_6[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_7 = handoffs_7.merge(filtered_df_final_x_7[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_8 = handoffs_8.merge(filtered_df_final_x_8[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')
handoffs_xy_9 = handoffs_9.merge(filtered_df_final_x_9[['gameId', 'playId', 'frameId', 'x', 'y']], left_on=['gameId', 'playId', 'frameId_x'], right_on=['gameId', 'playId', 'frameId'], how='inner').drop(columns='frameId')

handoffs_xy_1.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_2.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_3.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_4.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_5.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_6.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_7.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_8.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)
handoffs_xy_9.rename(columns={'x': 'bc_x', 'y': 'bc_y'}, inplace=True)

t_w1_pos = tracking_week_1[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w2_pos = tracking_week_2[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w3_pos = tracking_week_3[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w4_pos = tracking_week_4[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w5_pos = tracking_week_5[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w6_pos = tracking_week_6[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w7_pos = tracking_week_7[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w8_pos = tracking_week_8[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')
t_w9_pos = tracking_week_9[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']].merge(players[['nflId', 'position']], on='nflId', how='inner')

In [37]:
# group players by: DL, LB, DB, OL, TE, SK (skill for RB/WR), and QB

position_conditions_1 = [
    t_w1_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w1_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w1_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w1_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w1_pos['position'].isin(['TE']),
    t_w1_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w1_pos['position'].isin(['QB'])
]



position_conditions_2 = [
    t_w2_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w2_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w2_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w2_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w2_pos['position'].isin(['TE']),
    t_w2_pos['position'].isin(['RB', 'FB',  'WR']),
    t_w2_pos['position'].isin(['QB'])
]

position_conditions_3 = [
    t_w3_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w3_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w3_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w3_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w3_pos['position'].isin(['TE']),
    t_w3_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w3_pos['position'].isin(['QB'])
]


position_conditions_4 = [
    t_w4_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w4_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w4_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w4_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w4_pos['position'].isin(['TE']),
    t_w4_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w4_pos['position'].isin(['QB'])
]

position_conditions_5 = [
    t_w5_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w5_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w5_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w5_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w5_pos['position'].isin(['TE']),
    t_w5_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w5_pos['position'].isin(['QB'])
]

position_conditions_6 = [
    t_w6_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w6_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w6_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w6_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w6_pos['position'].isin(['TE']),
    t_w6_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w6_pos['position'].isin(['QB'])
]

position_conditions_7 = [
    t_w7_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w7_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w7_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w7_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w7_pos['position'].isin(['TE']),
    t_w7_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w7_pos['position'].isin(['QB'])
]

position_conditions_8 = [
    t_w8_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w8_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w8_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w8_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w8_pos['position'].isin(['TE']),
    t_w8_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w8_pos['position'].isin(['QB'])
]

position_conditions_9 = [
    t_w9_pos['position'].isin(['DT', 'DE', 'NT']),
    t_w9_pos['position'].isin(['CB', 'DB', 'FS', 'SS']),
    t_w9_pos['position'].isin(['OLB', 'ILB', 'MLB']),
    t_w9_pos['position'].isin(['T', 'G', 'C', 'OL']),
    t_w9_pos['position'].isin(['TE']),
    t_w9_pos['position'].isin(['RB', 'FB', 'WR']),
    t_w9_pos['position'].isin(['QB'])
]



position_choices = ['DL',
                    'DB',
                    'LB',
                    'OL',
                    'TE',
                    'SK',
                    'QB']

In [38]:
# reassign player positions to one of the 7 found in 'position_choices'

t_w1_pos['new_pos'] = np.select(position_conditions_1, position_choices, default='OTHER')
t_w2_pos['new_pos'] = np.select(position_conditions_2, position_choices, default='OTHER')
t_w3_pos['new_pos'] = np.select(position_conditions_3, position_choices, default='OTHER')
t_w4_pos['new_pos'] = np.select(position_conditions_4, position_choices, default='OTHER')
t_w5_pos['new_pos'] = np.select(position_conditions_5, position_choices, default='OTHER')
t_w6_pos['new_pos'] = np.select(position_conditions_6, position_choices, default='OTHER')
t_w7_pos['new_pos'] = np.select(position_conditions_7, position_choices, default='OTHER')
t_w8_pos['new_pos'] = np.select(position_conditions_8, position_choices, default='OTHER')
t_w9_pos['new_pos'] = np.select(position_conditions_9, position_choices, default='OTHER')

# _nb stands for "not ball carrier", so the x, y position of the player
t_w1_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w2_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w3_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w4_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w5_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w6_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w7_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w8_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)
t_w9_pos.rename(columns={'x': 'x_nb', 'y': 'y_nb','s': 's_nb','a': 'a_nb','dis': 'dis_nb','o': 'o_nb','dir': 'dir_nb', }, inplace=True)

In [39]:
handoffs_w1 = handoffs_xy_1.merge(t_w1_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w2 = handoffs_xy_2.merge(t_w2_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w3 = handoffs_xy_3.merge(t_w3_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w4 = handoffs_xy_4.merge(t_w4_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w5 = handoffs_xy_5.merge(t_w5_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w6 = handoffs_xy_6.merge(t_w6_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w7 = handoffs_xy_7.merge(t_w7_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w8 = handoffs_xy_8.merge(t_w8_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')
handoffs_w9 = handoffs_xy_9.merge(t_w9_pos, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'], how='inner')



handoffs_w_others_1 = handoffs_w1[(handoffs_w1['frameId'] >= handoffs_w1['frameId_x']) & (handoffs_w1['frameId'] <= handoffs_w1['frameId_y'])]
handoffs_w_others_2 = handoffs_w2[(handoffs_w2['frameId'] >= handoffs_w2['frameId_x']) & (handoffs_w2['frameId'] <= handoffs_w2['frameId_y'])]
handoffs_w_others_3 = handoffs_w3[(handoffs_w3['frameId'] >= handoffs_w3['frameId_x']) & (handoffs_w3['frameId'] <= handoffs_w3['frameId_y'])]
handoffs_w_others_4 = handoffs_w4[(handoffs_w4['frameId'] >= handoffs_w4['frameId_x']) & (handoffs_w4['frameId'] <= handoffs_w4['frameId_y'])]
handoffs_w_others_5 = handoffs_w5[(handoffs_w5['frameId'] >= handoffs_w5['frameId_x']) & (handoffs_w5['frameId'] <= handoffs_w5['frameId_y'])]
handoffs_w_others_6 = handoffs_w6[(handoffs_w6['frameId'] >= handoffs_w6['frameId_x']) & (handoffs_w6['frameId'] <= handoffs_w6['frameId_y'])]
handoffs_w_others_7 = handoffs_w7[(handoffs_w7['frameId'] >= handoffs_w7['frameId_x']) & (handoffs_w7['frameId'] <= handoffs_w7['frameId_y'])]
handoffs_w_others_8 = handoffs_w8[(handoffs_w8['frameId'] >= handoffs_w8['frameId_x']) & (handoffs_w8['frameId'] <= handoffs_w8['frameId_y'])]
handoffs_w_others_9 = handoffs_w9[(handoffs_w9['frameId'] >= handoffs_w9['frameId_x']) & (handoffs_w9['frameId'] <= handoffs_w9['frameId_y'])]

In [41]:
handoffs_w_others_1 = handoffs_w_others_1.merge(tracking_week_1[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_2 = handoffs_w_others_2.merge(tracking_week_2[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_3 = handoffs_w_others_3.merge(tracking_week_3[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_4 = handoffs_w_others_4.merge(tracking_week_4[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_5 = handoffs_w_others_5.merge(tracking_week_5[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_6 = handoffs_w_others_6.merge(tracking_week_6[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_7 = handoffs_w_others_7.merge(tracking_week_7[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_8 = handoffs_w_others_8.merge(tracking_week_8[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)
handoffs_w_others_9 = handoffs_w_others_9.merge(tracking_week_9[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
            left_on=['frameId', 'ballCarrierId', 'gameId', 'playId'],
                          right_on=['frameId', 'nflId', 'gameId', 'playId']).drop(['nflId_y', 'bc_x', 'bc_y'], axis=1)

In [42]:
handoffs_w_others_1 = handoffs_w_others_1.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_2 = handoffs_w_others_2.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_3 = handoffs_w_others_3.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_4 = handoffs_w_others_4.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_5 = handoffs_w_others_5.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_6 = handoffs_w_others_6.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_7 = handoffs_w_others_7.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_8 = handoffs_w_others_8.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})
handoffs_w_others_9 = handoffs_w_others_9.rename(columns={"x": "bc_x", "y": "bc_y", "nflId_x": "nflId"})

### Calculate each player's distance and angle to the ball carrier

In [44]:
import math

# Function to calculate Euclidean distance
def calculate_distance(row):
    return ((row['bc_x'] - row['x_nb']) ** 2 + (row['bc_y'] - row['y_nb']) ** 2) ** 0.5


def normalize_angle(angle):
  return angle % 360

# function to calculate the angle
def calculate_angle(row):
    # Calculate the differences in coordinates
    delta_x = row['x_nb'] - row['bc_x']
    delta_y = row['y_nb'] - row['bc_y']

    # Calculate the angle in radians
    angle_rad = math.atan2(delta_y, delta_x)

    # Convert the angle to degrees (optional)
    angle_deg = math.degrees(angle_rad)

    if row['playDirection'] == 'left':
      angle_deg = (angle_deg + 180) % 360

    return normalize_angle(angle_deg)


In [45]:
# Calculate distance and angle to the ball carrier using the previous functions

handoffs_w_others_1['dist_to_ball'] = handoffs_w_others_1.apply(calculate_distance, axis=1)
handoffs_w_others_2['dist_to_ball'] = handoffs_w_others_2.apply(calculate_distance, axis=1)
handoffs_w_others_3['dist_to_ball'] = handoffs_w_others_3.apply(calculate_distance, axis=1)
handoffs_w_others_4['dist_to_ball'] = handoffs_w_others_4.apply(calculate_distance, axis=1)
handoffs_w_others_5['dist_to_ball'] = handoffs_w_others_5.apply(calculate_distance, axis=1)
handoffs_w_others_6['dist_to_ball'] = handoffs_w_others_6.apply(calculate_distance, axis=1)
handoffs_w_others_7['dist_to_ball'] = handoffs_w_others_7.apply(calculate_distance, axis=1)
handoffs_w_others_8['dist_to_ball'] = handoffs_w_others_8.apply(calculate_distance, axis=1)
handoffs_w_others_9['dist_to_ball'] = handoffs_w_others_9.apply(calculate_distance, axis=1)

handoffs_w_others_1['ang_to_ball'] = handoffs_w_others_1.apply(calculate_angle, axis=1)
handoffs_w_others_2['ang_to_ball'] = handoffs_w_others_2.apply(calculate_angle, axis=1)
handoffs_w_others_3['ang_to_ball'] = handoffs_w_others_3.apply(calculate_angle, axis=1)
handoffs_w_others_4['ang_to_ball'] = handoffs_w_others_4.apply(calculate_angle, axis=1)
handoffs_w_others_5['ang_to_ball'] = handoffs_w_others_5.apply(calculate_angle, axis=1)
handoffs_w_others_6['ang_to_ball'] = handoffs_w_others_6.apply(calculate_angle, axis=1)
handoffs_w_others_7['ang_to_ball'] = handoffs_w_others_7.apply(calculate_angle, axis=1)
handoffs_w_others_8['ang_to_ball'] = handoffs_w_others_8.apply(calculate_angle, axis=1)
handoffs_w_others_9['ang_to_ball'] = handoffs_w_others_9.apply(calculate_angle, axis=1)

**- D) Rank the players for each frame of a play in the following manner: 1) ball carrier, 2-8) 7 closest defenders to the ball carrier, 9-15) 7 closest offensive players to the ball carrier (besides QB), 16-22) remaining players distance to the ball carrier**

In [47]:
# formula to rank players by 1) ball carrier, 2-8) 7 closest defenders to the ball carrier in order, 9-15) 7 closest
# offensive players (not including QB) to the ball carrier, 16-22) remaining players

def custom_sort(group):
    # Avoid unnecessary copying if possible
    group['rank'] = np.nan

    # Rank the ball carrier as 1
    group.loc[group['ballCarrierId'] == group['nflId'], 'rank'] = 1

    # Identify defenders and offensive players
    defenders = group['new_pos'].isin(['DL', 'LB', 'DB'])
    offensive_players = group['new_pos'].isin(['OL', 'SK', 'TE']) & (group['dist_to_ball'] > 0)

    # Sort defenders by distance to the ball and assign ranks 2 to 8
    defenders_sorted_indices = group.loc[defenders, 'dist_to_ball'].nsmallest(7).index
    group.loc[defenders_sorted_indices, 'rank'] = range(2, 9)

    # Sort offensive players by distance to the ball and assign ranks 9 to 15
    offensive_sorted_indices = group.loc[offensive_players, 'dist_to_ball'].nsmallest(7).index
    group.loc[offensive_sorted_indices, 'rank'] = range(9, 16)

    # Fill in ranks for any remaining players
    remaining_indices = group[group['rank'].isna()].index
    group.loc[remaining_indices, 'rank'] = range(16, len(remaining_indices) + 16)

    return group

**E) Saved as a .pkl file in Google Drive to be used in "DataCleaning_2"**

Create sorted_df_(1-9) and save for future use

In [ ]:
sorted_df_1 = handoffs_w_others_1.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_2 = handoffs_w_others_2.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_3 = handoffs_w_others_3.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_4 = handoffs_w_others_4.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_5 = handoffs_w_others_5.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_6 = handoffs_w_others_6.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_7 = handoffs_w_others_7.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_8 = handoffs_w_others_8.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)
sorted_df_9 = handoffs_w_others_9.groupby(['gameId', 'playId', 'frameId']).apply(custom_sort).reset_index(drop=True)

In [ ]:
# data begins at frame when ball carrier gets the ball and ends when he gets tackled, fumbles, scores touchdown, or goes OB
# The players are also ranked according to the function above, and shows their distance and angle to ball carrier

sorted_df_1 = sorted_df_1.merge(tracking_week_1[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_2 = sorted_df_2.merge(tracking_week_2[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_3 = sorted_df_3.merge(tracking_week_3[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_4 = sorted_df_4.merge(tracking_week_4[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_5 = sorted_df_5.merge(tracking_week_5[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_6 = sorted_df_6.merge(tracking_week_6[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_7 = sorted_df_7.merge(tracking_week_7[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_8 = sorted_df_8.merge(tracking_week_8[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])
sorted_df_9 = sorted_df_9.merge(tracking_week_9[['x', 'y', 'nflId', 'gameId', 'playId', 'frameId']],
                  left_on=['ballCarrierId', 'gameId', 'playId', 'frameId'], right_on=['nflId', 'gameId', 'playId', 'frameId'])

ball carrier = 'bc' and non ball carrier = 'nb'

In [51]:
sorted_df_1.head(5).T

,0,1,2,3,4
gameId,2022090800,2022090800,2022090800,2022090800,2022090800
playId,101,101,101,101,101
frameId_x,19,19,19,19,19
event_x,handoff,handoff,handoff,handoff,handoff
frameId_y,45,45,45,45,45
event_y,tackle,tackle,tackle,tackle,tackle
ballCarrierId,47857,47857,47857,47857,47857
ballCarrierDisplayName,Devin Singletary,Devin Singletary,Devin Singletary,Devin Singletary,Devin Singletary
playDirection,left,left,left,left,left
nflId,35472.0,38577.0,41239.0,42392.0,42816.0


In [ ]:
# Saved these files for future use

# sorted_df_1.to_pickle('sorted_df_1.pkl')
# sorted_df_2.to_pickle('sorted_df_2.pkl')
# sorted_df_3.to_pickle('sorted_df_3.pkl')
# sorted_df_4.to_pickle('sorted_df_4.pkl')
# sorted_df_5.to_pickle('sorted_df_5.pkl')
# sorted_df_6.to_pickle('sorted_df_6.pkl')
# sorted_df_7.to_pickle('sorted_df_7.pkl')
# sorted_df_8.to_pickle('sorted_df_8.pkl')
# sorted_df_9.to_pickle('sorted_df_9.pkl')